In [1]:
import pandas as pd
import datetime
import pytz
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)


# Function to resample the 1 minute timeframe data to new timeframe
def resample_data(data, timeframe):
    
    resampled_data = data.resample(timeframe).agg({
        'Open': 'first',
        'High': 'max',
        'Low': 'min',
        'Close': 'last'
    }).dropna()
    
    return resampled_data


# Function to find high and low prices for specific time ranges within a day
def find_high_low(df, time_ranges):
    results = []
    daily_data = df.resample('D')

    for date, group in daily_data:
        result = {'Date': date.date(), 'Day of the Week': date.strftime('%A')}

        for start_time, end_time in time_ranges:
            intraday_group = group.between_time(start_time, end_time)
            if not intraday_group.empty:
                result[f'{start_time}-{end_time} High'] = intraday_group['High'].max()
                result[f'{start_time}-{end_time} Low'] = intraday_group['Low'].min()

        
        results.append(result)

    # Create DataFrame and drop rows with 'Friday' as Day of the Week
    results_df = pd.DataFrame(results)
    results_df = results_df.rename(columns={'19:30-20:30 High': 'Opening Range High', '19:30-20:30 Low': 'Opening Range Low',
                                            '20:30-16:00 High': 'Daily High', '20:30-16:00 Low': 'Daily Low'
                                           })
    results_df['Initial Range Size ($)'] = results_df['Opening Range High'] - results_df['Opening Range Low']
    results_df = results_df.dropna(subset=['Opening Range High'])
    results_df = results_df.reset_index()
    return results_df




# Function to find the first candle closing above or below the high and low after the 19:30-20:30 time range
def find_first_candle(df, time_ranges):
    high_low_data = find_high_low(df, time_ranges)
    
    results = []
    
    # Get the first time range's end time
    _, first_end_time = time_ranges[0]
    
    for _, row in high_low_data.iterrows():
        date = row['Date']
        day_group = df.loc[df.index.date == date]
        
        # Filter data to only include times after the end of the first time range
        day_group = day_group.between_time(first_end_time, '23:59:59')
        
        # Check for the first candle that crosses the 19:30-20:30 High or Low
        for idx, candle in day_group.iterrows():
            condition = None
            if candle['Close'] > row['Opening Range High']:
                condition = 'Above Opening High'
            elif candle['Close'] < row['Opening Range Low']:
                condition = 'Below Opening Low'
            
            if condition:
                results.append({
                    'Date': idx.date(),
                    'First Candle Close Outside of Range Time': idx.time(),
                    'First Candle Close Condition': condition,
                    'First Candle Close Price': candle['Close'],
                    'Opening Range High': row['Opening Range High'],
                    'Opening Range Low': row['Opening Range Low'],
                    'Daily High': row['Daily High'],
                    'Daily Low': row['Daily Low']
                })
                break

    return pd.DataFrame(results)


# Function to find the first candle fully closing above or below the high and low after the 19:30-20:30 time range
def find_first_candle_outside_range(df, time_ranges):
    high_low_data = find_high_low(df, time_ranges)
    
    results = []
    
    # Get the first time range's end time
    _, first_end_time = time_ranges[0]
    
    for _, row in high_low_data.iterrows():
        date = row['Date']
        day_group = df.loc[df.index.date == date]
        
        # Filter data to only include times after the end of the first time range
        day_group = day_group.between_time(first_end_time, '23:59:59')
        
        # Check for the first candle where open, high, low, and close are all outside the 19:30-20:30 High or Low
        for idx, candle in day_group.iterrows():
            condition = None
            if (candle['High'] < row['Opening Range Low'] or candle['Low'] > row['Opening Range High']):
                if candle['Close'] > row['Opening Range High']:
                    condition = 'Above Opening High'
                elif candle['Close'] < row['Opening Range Low']:
                    condition = 'Below Opening Low'
            
            if condition:
                results.append({
                    'Date': idx.date(),
                    'First Candle Close Fully Outside of Open Range Time': idx.time(),
                    'First Fully Close Condition': condition,
                    'First Fully Close Price': candle['Close'],
                    'Opening Range High': row['Opening Range High'],
                    'Opening Range Low': row['Opening Range Low'],
                    'Daily High': row['Daily High'],
                    'Daily Low': row['Daily Low']
                })
                break

    return pd.DataFrame(results)

# Function to track if price crosses the opposite bound after the first candle
def find_opposite_cross(df, time_ranges):
    first_candle_data = find_first_candle(df, time_ranges)
    results = []

    for _, row in first_candle_data.iterrows():
        date = row['Date']
        first_candle_time = row['First Candle Close Outside of Range Time']
        condition = row['First Candle Close Condition']
        high = row['Opening Range High']
        low = row['Opening Range Low']

        next_day = pd.to_datetime(date) + pd.Timedelta(days=1)
        start_range = pd.Timestamp.combine(date, first_candle_time).tz_localize('America/New_York')
        end_range = pd.Timestamp.combine(next_day, pd.Timestamp('16:00').time()).tz_localize('America/New_York')
       
        # Filter for the time range from the first candle until 4 PM the next day
        time_filtered_group = df[(df.index >= start_range) & (df.index <= end_range)]
        
        # Check for the first cross to the opposite side
        if condition == 'Above Opening High':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] <= low:
                    results.append({
                        'Date of Cross': idx.date(),
                        'Cross of Opening Range Closing Time': idx.time(),
                        'Cross of Opening Range Low': 'Crossed Below Opening Range Low',
                        'Cross of Opening Range Low Price': candle['Close'],
                    })
                    break
        elif condition == 'Below Opening Low':
            for idx, candle in time_filtered_group.iterrows():
                if candle['Close'] >= high:
                    results.append({
                        'Date of Cross': idx.date(),
                        'Cross of Opening Range Closing Time': idx.time(),
                        'Cross of Opening Range High': 'Crossed Above Opening Range High',
                        'Cross of Opening Range High Price': candle['Close'],
                    })
                    break

    return pd.DataFrame(results)


def convert_column_to_nyc_time(df, datetime_column):
    # Define New York City timezone
    nyc_tz = pytz.timezone('America/New_York')

    # Convert each datetime in the specified column to New York City time
    df[datetime_column] = pd.to_datetime(df[datetime_column])  # Ensure the column is in datetime format
    df[datetime_column] = df[datetime_column].apply(lambda x: x.astimezone(nyc_tz))
    
    # Example
    # df = convert_column_to_nyc_time(df, 'Datetime')

    return df


In [2]:
# All Futures assets currently downloaded for
TickerList = (
    "MES=F", "MYM=F", "NG=F", "MNQ=F", 
    "ES=F", "NQ=F", "YM=F", "CL=F", 
    "MCL=F", "GC=F", "MGC=F", "RTY=F", 
    "M2K=F", "SI=F", "PL=F", "HG=F", "SIL=F"
    )
# Ticksize in $ amounts minimum movement
Profit_per_Tick = {
    "MES=F" :  1.25,    "MYM=F" :  0.50,    "NG=F"  : 10.00,    "MNQ=F" :  0.50,
    "ES=F"  : 12.50,    "NQ=F"  :  5.00,    "YM=F"  :  5.00,    "CL=F"  : 10.00,
    "MCL=F" :  1.00,    "GC=F"  : 10.00,    "MGC=F" :  1.00,    "RTY=F" :  5.00,
    "M2K=F" :  0.50,    "SI=F"  : 25.00,    "PL=F"  :  5.00,    "HG=F"  : 12.50,
    "SIL=F" : 10.00
    }

# Ticksize in minimum ticks
TickSize = {
    "MES=F" :   0.2500,   "MYM=F" :   1.0000,   "NG=F"  :   0.0010,   "MNQ=F" :   0.2500,
    "ES=F"  :   0.2500,   "NQ=F"  :   0.2500,   "YM=F"  :   1.0000,   "CL=F"  :   0.0100,
    "MCL=F" :   0.0100,   "GC=F"  :   0.1000,   "MGC=F" :   0.1000,   "RTY=F" :   0.1000,
    "M2K=F" :   0.1000,   "SI=F"  :   0.0050,   "PL=F"  :   0.1000,   "HG=F"  :   0.0005,
    "SIL=F" :   0.0100
    }


In [3]:
Ticks_per_Point = {}

# Calculate the multiplier for the number of number of ticks in a point
for asset, ticksize in TickSize.items():
    multiplier = 1 / ticksize
    Ticks_per_Point[asset] = multiplier


In [6]:
for ticker in TickerList:
    df = pd.read_csv(f'Futures Asset Data/{ticker.split("=")[0]}.csv', index_col=False)
    df = df.drop(columns=['Adj Close', 'Volume'])
    df = pd.DataFrame(df)
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)

    # Resample on 5 minute and 15 minute timeframes.
    # df = resample_data(df, '5min')
    df = resample_data(df, '15min')
    
    # Define the two time ranges
    time_ranges = [('19:30', '20:30'), ('20:30', '16:00')]

    df1 = find_high_low(df, time_ranges)
    df1['Range Size (ticks)'] = df1['Initial Range Size ($)'] / TickSize[ticker]
    df1['Range Size (points)'] = df1['Initial Range Size ($)'] / Ticks_per_Point[ticker]
        
    df2 = find_first_candle(df, time_ranges)
    df2['Max Distribution/Drawdown (points)'] = None
    df2['Max Distribution (%)'] = None

    df2.loc[df2['First Candle Close Condition'] == 'Above Opening High', 'Max Distribution/Drawdown (points)'] = (
        df2['Daily High'] - df2['First Candle Close Price']) / Ticks_per_Point[ticker] 

    df2.loc[df2['First Candle Close Condition'] == 'Below Opening Low', 'Max Distribution/Drawdown (points)'] = (
        df2['First Candle Close Price'] - df2['Daily Low']) / Ticks_per_Point[ticker]

    df2.loc[df2['First Candle Close Condition'] == 'Above Opening High', 'Max Distribution (%)'] = ((
        df2['Daily High'] - df2['First Candle Close Price']) / df2['Daily High']) * 100

    df2.loc[df2['First Candle Close Condition'] == 'Below Opening Low', 'Max Distribution (%)'] =((
        df2['First Candle Close Price'] - df2['Daily Low']) / df2['Daily Low']) * 100


    df3 = find_first_candle_outside_range(df, time_ranges)
    df3['Max Distribution/Drawdown (Fully Outside) (points)'] = None
    df3['Max Distribution (Fully Outside) (%)'] = None

    df3.loc[df3['First Fully Close Condition'] == 'Above Opening High', 'Max Distribution/Drawdown (Fully Outside) (points)'] = (
        df3['Daily High'] - df3['First Fully Close Price']) / Ticks_per_Point[ticker]

    df3.loc[df3['First Fully Close Condition'] == 'Below Opening Low', 'Max Distribution/Drawdown (Fully Outside) (points)'] = (
        df3['First Fully Close Price'] - df3['Daily Low']) / Ticks_per_Point[ticker]

    df3.loc[df3['First Fully Close Condition'] == 'Above Opening High', 'Max Distribution (Fully Outside) (%)'] = ((
        df3['Daily High'] - df3['First Fully Close Price']) / df2['Daily High']) * 100

    df3.loc[df3['First Fully Close Condition'] == 'Below Opening Low', 'Max Distribution (Fully Outside) (%)'] =((
        df3['First Fully Close Price'] - df3['Daily Low']) / df3['Daily Low']) * 100

    df4 = find_opposite_cross(df, time_ranges)

    combined_df12 = pd.merge(df1, df2, left_index=True, right_index=True)
    combined_df12 = combined_df12.drop(columns=['Date_y', 'Opening Range High_y', 'Opening Range Low_y', 'Daily High_y',
                                     'Daily Low_y'])
    combined_df12 = combined_df12.rename(columns={'Date_x': 'Date', 'Opening Range High_x': 'Opening Range High',
                                                'Opening Range Low_x': 'Opening Range Low', 'Daily High_x': 'Daily High',
                                                'Daily Low_x': 'Daily Low'
                                               })

    combined_df123 = pd.merge(combined_df12, df3, left_index=True, right_index=True)
    combined_df123 = combined_df123.drop(columns=['Date_y', 'Opening Range High_y', 'Opening Range Low_y', 'Daily High_y',
                                                  'Daily Low_y'])
    combined_df123 = combined_df123.rename(columns={'Date_x': 'Date', 'Opening Range High_x': 'Opening Range High',
                                                    'Opening Range Low_x': 'Opening Range Low', 'Daily High_x': 'Daily High',
                                                    'Daily Low_x': 'Daily Low'
                                                   })

    combined_df1234 = pd.merge(combined_df123, df4, left_index=True, right_index=True)
    combined_df1234 = combined_df1234.drop(columns=['index'])
    
    combined_df1234['Win/(Loss)'] = 'Loss'
    combined_df1234.loc[(combined_df1234['First Candle Close Condition'] == 'Above Opening High') & \
        (combined_df1234['Cross of Opening Range Low'] == 'Crossed Below Opening Range Low'), 'Win/(Loss)'] = 'Win'
    
    combined_df1234.loc[(combined_df1234['First Candle Close Condition'] == 'Below Opening Low') & \
        (combined_df1234['Cross of Opening Range High'] == 'Crossed Above Opening Range High'), 'Win/(Loss)'] = 'Win'


    combined_df1234 = combined_df1234[['Date', 'Day of the Week', 'Win/(Loss)', 'Opening Range High', 'Opening Range Low',
                                       'Daily High', 'Daily Low', 'Initial Range Size ($)', 'Range Size (ticks)',
                                       'Range Size (points)', 'First Candle Close Outside of Range Time', 'First Candle Close Price',
                                       'First Candle Close Condition', 'First Candle Close Fully Outside of Open Range Time',
                                       'First Fully Close Price', 'First Fully Close Condition', 'Max Distribution (%)',
                                       'Max Distribution/Drawdown (points)', 'Max Distribution (Fully Outside) (%)',
                                       'Max Distribution/Drawdown (Fully Outside) (points)', 'Date of Cross',
                                       'Cross of Opening Range Closing Time', 'Cross of Opening Range Low',
                                       'Cross of Opening Range Low Price', 'Cross of Opening Range High',
                                       'Cross of Opening Range High Price']]

    # Write the .csv files
    # combined_df1234.to_csv(f'Backtest Results/1 Minute Timeframe Results/{ticker.split('=')[0]}_1min.csv', index=False)
    # combined_df1234.to_csv(f'Backtest Results/5 Minute Timeframe Results/{ticker.split('=')[0]}_5min.csv', index=False)
    combined_df1234.to_csv(f'Backtest Results/15 Minute Timeframe Results/{ticker.split('=')[0]}_15min.csv', index=False)

    print(ticker)

print('Done')

MES=F
MYM=F
NG=F
MNQ=F
ES=F
NQ=F
YM=F
CL=F
MCL=F
GC=F
MGC=F
RTY=F
M2K=F
SI=F
PL=F
HG=F
SIL=F
Done


In [10]:
for ticker in TickerList:
    # df = pd.read_csv(f'Backtest Results/1 Minute Timeframe Results/{ticker.split("=")[0]}_1min.csv', index_col=False)
    # df = pd.read_csv(f'Backtest Results/5 Minute Timeframe Results/{ticker.split("=")[0]}_5min.csv', index_col=False)
    df = pd.read_csv(f'Backtest Results/15 Minute Timeframe Results/{ticker.split("=")[0]}_15min.csv', index_col=False)

    plt.scatter(df.index, df['Max Distribution (%)'], color='blue', label='Max Distribution (%)')
    plt.scatter(df.index, df['Max Distribution (Fully Outside) (%)'], color='green', label='Max Distribution (Fully Outside) (%)')
    
    # Set the y-axis range based on the maximum value from both columns
    y_max = max(df['Max Distribution (%)'].max(), df['Max Distribution (Fully Outside) (%)'].max())
    plt.ylim(0, y_max)
    
    # Add labels, title, and legend
    plt.xlabel('Index')
    plt.ylabel('Max MAE/Drawdown')
    # plt.title('Scatter Plot of Two Columns')
    plt.legend()
    
    # Save the plot
    # plt.savefig(f'Backtest Results/1 Minute Timeframe Results/{ticker.split('=')[0]}_1min.png')
    # plt.savefig(f'Backtest Results/5 Minute Timeframe Results/{ticker.split('=')[0]}_5min.png')
    plt.savefig(f'Backtest Results/15 Minute Timeframe Results/{ticker.split('=')[0]}_15min.png')


    print(ticker)
    plt.clf()

print('Done')

MES=F
MYM=F
NG=F
MNQ=F
ES=F
NQ=F
YM=F
CL=F
MCL=F
GC=F
MGC=F
RTY=F
M2K=F
SI=F
PL=F
HG=F
SIL=F
Done


<Figure size 640x480 with 0 Axes>